I think it would be neat just to examine some of the musical preference interactions and the questions/polls information. With that in  mind this can be just a walk through notebook and maybe at the end well look at some modeling.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
cols = pd.read_csv('../input/columns.csv')
cols.head()

In [ ]:
data = pd.read_csv('../input/responses.csv')
data.head()

In [ ]:
cnt = 0
for c in data.columns:
    print(c, cnt, end=',')
    cnt += 1

In [ ]:
music = data.iloc[:,0:18]
quesOrPoll = data.iloc[:,131:]

In [ ]:
df = pd.concat([music, quesOrPoll], axis=1, join='outer')
df.columns
df_obj = df.select_dtypes(include=['object'])
for c in df_obj:
    df[c] = pd.factorize(df_obj[c])[0]

In [7]:
from collections import Counter
print(Counter(df_obj['Gender']))
print(Counter(df['Gender']))

There are probably many more pythonic and efficient ways to do what I did above but I like to step by step _day by day_ it.

In [ ]:
fig, ax=plt.subplots(1,1,figsize=(12,10))
sns.heatmap( df.corr(), ax=ax)

So we have a heat map here of the correlation matrix for my music and preference data frame that put together. There are some obvious correlations in there, i.e. Rock/Punk, Folk/Country, Height/Weight, etc. But also some non-obvious ones. Like Classical and Folk? 

Also why are Musicals and Latino Music negatively correlated with Height and Weight?

In [ ]:
df['binnedHeight'] = pd.cut(df['Height'], bins=8)
df['binnedWeight'] = pd.cut(df['Weight'], bins=8)

In [ ]:
fig, ax=plt.subplots(1,1,figsize=(12,8))
sns.violinplot(x='binnedWeight',y='Musical',hue='Gender',data=df,inner='quart')

This may have been a weird plot to go with for this instance, or perhaps a weird way to look at this data? With 0 as Female and 1 as Male, does this say that heavier males tend to dislike musical numbers?

In [11]:
fig, ax=plt.subplots(1,1,figsize=(12,8))
sns.violinplot(x='binnedWeight',y='Latino',hue='Gender',data=df,inner='quart')

This makes me think that generally the average person enjoys Latino music averagely.

Lets take a gander at things from a Gender perspective.

In [ ]:
grp = df.groupby('Gender')
gender = grp.agg('count')
gender.reset_index(inplace=True)
gender.drop(['Music','binnedHeight','binnedWeight','Gender'], axis=1, inplace=True)
gender.drop([0], axis=0, inplace=True)
gender.head()

In [27]:
gT = gender.T
color2 = iter(plt.cm.rainbow(np.linspace(0,1,len(gT))))
fig, ax=plt.subplots(1,2,figsize=(12,8),sharey=True)
ax=ax.ravel()
for j in range(len(gT)):
    col2 = next(color2);
    _ = ax[0].barh(j, gT.iloc[j,0], color=col2, align='center');
    ax[0].set_title('Female'); ax[0].set_xlabel('Counts');
    _ = ax[1].barh(j, gT.iloc[j,1], color=col2, align='center');
    ax[1].set_title('Male'); ax[0].set_xlabel('Counts');
plt.yticks(np.arange(len(gT.index)), gT.index, fontsize=9);

Thats a pretty even distribution of count data over all. 

I think we will stop here for now...